<a href="https://colab.research.google.com/github/450586509/practical-ml/blob/master/bert_sentence_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==3.3.0

AttributeError: ignored

In [2]:
#!pip install transformers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import torch
import transformers as tfs
import warnings
import torch.utils.data as Data
from sklearn import preprocessing
from sklearn.utils import shuffle

warnings.filterwarnings('ignore')

In [3]:
!curl -LO http://129.204.205.246/similarity_train.csv
!curl -LO http://129.204.205.246/similarity_test.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13.9M  100 13.9M    0     0   110k      0  0:02:09  0:02:09 --:--:--  123k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1577k  100 1577k    0     0  94158      0  0:00:17  0:00:17 --:--:-- 95010


In [3]:
test_file = "./similarity_test.csv"
train_file="./similarity_train.csv"
train_df = pd.read_csv(train_file, delimiter='\t', header=None)
train_df.columns = ["event_name","text1","text2","label"]
test_df = pd.read_csv(test_file, delimiter='\t', header=None)
test_df.columns = ["event_name","text1","text2","label"]

In [4]:
train_df.head()

,event_name,text1,text2,label
0,重大合同,公司第四届董事会第十二次临时会议审议通过《关于公司之全资子公司拟与关连人士签订施工合同协议的议案》,许月红获委任爲该公司副总经理,0
1,业绩利好,苏博特：上半年净利润同比预增20%到30%苏博特（603916）7月3日晚间公告,A股异动丨天味食品涨停上半年净利预增82.85%到101.18%,0
2,新增产能,週期底部扩产弹性最大,华侨城通过回购专用证券账户,0
3,公司订单,通达股份先后两次发布预中标国家电网项目提示性公告,中标金额约为人民币1.73亿元,1
4,股权激励,《多氟多化工股份有限公司2020年限制性股票激励计划(草案)》规定的首次授予条件已成就,格隆汇7月28日丨厦门信达披露2020年限制性股票激励计划(草案),1


### 合并两列

In [5]:
bert_sep = "[SEP]"
train_df["text"] = train_df["text1"] + bert_sep + train_df["text2"]
test_df["text"] = test_df["text1"] + bert_sep + test_df["text2"]

### 新增len列

In [6]:
print(train_df.shape)
train_df["len"] = train_df.text.apply((lambda x: len(x)))
test_df["len"] = test_df.text.apply((lambda x: len(x)))
print(train_df.shape)
#print(train_df.head)
for index, row in train_df.iterrows():
  print(f"""index={index} text={row.get("text")} len={len(row.get("text"))}""")
  if index > 5:
    break

(83160, 5)
(83160, 6)
index=0 text=公司第四届董事会第十二次临时会议审议通过《关于公司之全资子公司拟与关连人士签订施工合同协议的议案》[SEP]许月红获委任爲该公司副总经理 len=68
index=1 text=苏博特：上半年净利润同比预增20%到30%苏博特（603916）7月3日晚间公告[SEP]A股异动丨天味食品涨停上半年净利预增82.85%到101.18% len=77
index=2 text=週期底部扩产弹性最大[SEP]华侨城通过回购专用证券账户 len=28
index=3 text=通达股份先后两次发布预中标国家电网项目提示性公告[SEP]中标金额约为人民币1.73亿元 len=44
index=4 text=《多氟多化工股份有限公司2020年限制性股票激励计划(草案)》规定的首次授予条件已成就[SEP]格隆汇7月28日丨厦门信达披露2020年限制性股票激励计划(草案) len=81
index=5 text=公司所属部分子公司拟与宁夏建材所属赛马物联科技(宁夏)有限公司(以下简称“赛马物联”)、乌海市西水水泥有限责任公司(以下简称“乌海西水”)签署网络货运服务及熟料采购合同[SEP]电动车企业宝马、奔驰、特斯拉等为保证供应链安全纷纷签订合同 len=118
index=6 text=评级「买入」[SEP]重申中国中铁“买入”评级 len=23


In [7]:
train_df = train_df[train_df.len < 500]
test_df = test_df[test_df.len < 500]
print(f"""max_len={train_df["len"].max()}""")
print(f"""mean_len={train_df["len"].mean()}""")
print(f"shape={train_df.shape}")
print(train_df.head())
print(test_df.head())

max_len=199
mean_len=63.958585858585856
shape=(83160, 6)
  event_name  ... len
0       重大合同  ...  68
1       业绩利好  ...  77
2       新增产能  ...  28
3       公司订单  ...  44
4       股权激励  ...  81

[5 rows x 6 columns]
  event_name  ... len
0       股份回购  ...  67
1       增持评级  ...  97
2       高管任职  ...  25
3     产品销量上升  ...  47
4       公司订单  ...  87

[5 rows x 6 columns]


In [8]:
event_counts = train_df.event_name.value_counts()
print(f"type of event_counts = {type(event_counts)}")
print(event_counts)
print(f"样本最少为{event_counts.min()}")
print(f"样本最多为{event_counts.max()}")
print(f"样本平均为{round(event_counts.mean(),4)}")

type of event_counts = <class 'pandas.core.series.Series'>
业绩预增      4532
业绩利好      4525
股市利好      4520
增持加仓      4517
重大合同      4517
买入评级      4517
生产投产      4507
高管任职      4504
股份回购      4503
解除质押      4473
推荐评级      4470
股权激励      4468
对外合作      4466
公司订单      4462
增持评级      4460
新增产能      3202
评级上调      2988
收购并购      2710
优于大市      2307
价格上涨      2264
产品销量上升    2248
Name: event_name, dtype: int64
样本最少为2248
样本最多为4532
样本平均为3960.0


In [9]:
model_path = "bert-base-chinese"
tokenizer = tfs.BertTokenizer.from_pretrained(model_path)
train_tokenized = train_df.text.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
print(f"train_tokenized type = {type(train_tokenized)}")
train_max_len = 0
for i in train_tokenized.values:
    if len(i) > train_max_len:
        train_max_len = len(i)

train_tokenized type = <class 'pandas.core.series.Series'>


In [10]:
x="上升[SEP]公司"
#tokenizer.encode_plus(x,)
t1 = "上升"
t2 = "公司"
max_len=10
x1,x2 = x.split("[SEP]")
inputs = tokenizer.encode_plus(
            x1,
            x2,
            add_special_tokens=True,
            max_length=max_len,
            pad_to_max_length=True,
            truncation=True,
            return_token_type_ids=True
            )
print(inputs)

{'input_ids': [101, 677, 1285, 102, 1062, 1385, 102, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 1, 1, 1, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 0, 0, 0]}


In [11]:
y = train_df.label.tolist()
test_y = test_df.label.tolist()

x = train_df.text.tolist()
x,y = shuffle(x,y)
test_x = test_df.text.tolist()
for i,text in enumerate(x):
    print(text)
    if i >=5:
        break
for i , text in enumerate(y):
    print(text)
    if i>=5:
        break
x = np.array(x)
test_x = np.array(test_x)
test_y = np.array(test_y)
y = np.array(y)
print(type(y))
print(y[0:10])
print(f"x shape ={x.shape}")
print(f"y shape ={y.shape}")

大和将玖龙纸业（02689）的评级由“优于大市”上调至“买入”[SEP]尚未与招标人签订正式合同
首予「增持」评级[SEP]《大行报告》摩通下调邮储银行目标价至5.5元评级「增持」
长安汽车、中兴通讯、恒生电子、闻泰科技、紫光股份、美亚柏科、中科创达、海格通信、卫宁健康等9只个股均受到机构5次及以上给予“买入”或“增持”等看好评级[SEP]并维持“增持”股票评级
华谊腾讯娱乐获执行董事袁海波在场内以每股均价0.1123港元增持2000万股[SEP]中国燃气获主要股东邱达强在场内以每股均价25.6063港元增持62.12万股
【经营业绩】唐人神：上半年净利润4.32亿元同比增长859%唐人神（002567）7月26日晚间披露半年报[SEP]公司控股子公司四川华都核设备制造有限公司(“华都公司”)与中核龙原科技有限公司(“中核龙原”)签署了《示范快堆2号机组一体化扩散型氢计采购合同》
推动实现公司销售业绩上升[SEP]三峡水利（600116）公司投产的水电装机容量共计26.98万千瓦
0
1
1
1
0
0
<class 'numpy.ndarray'>
[0 1 1 1 0 0 1 1 0 0]
x shape =(83160,)
y shape =(83160,)


In [12]:
MAX_LEN = train_max_len
print(f"max_len={MAX_LEN}")
TRAIN_BATCH_SIZE = 12
VALID_BATCH_SIZE = 8
EPOCHS = 1
LEARNING_RATE = 1e-05
#print(f"max_len={MAX_LEN}")

max_len=195


In [13]:
class_number=1
print(f"class_number={class_number}")
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert = tfs.BertModel.from_pretrained(model_path)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, class_number)
        #self.out_act = nn.Sigmoid()
    
    def forward(self, ids, mask, token_type_ids,return_dict=True):
        outputs= self.bert(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=True)
        print(type(outputs))
        print(outputs)
        hidden_state = outputs.hidden_states
        class_vec = outputs.pooler_output
        output_2 = self.dropout(class_vec)
        logits = self.linear(output_2)
        #hidden_average = torch.mean(hidden_state, 1)
        #sentence_vec = hidden_average[0]
        return logits,_

model = BERTClass()
model.to(device)

class_number=1


BERTClass(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  

In [14]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [15]:
#y=ont_hot
#x:文本数据。
class CustomDataset(Dataset):

    def __init__(self, x, y, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.comment_text = x
        self.targets = y
        self.max_len = max_len

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, index):
        comment_text = str(self.comment_text[index])
        #comment_text = " ".join(comment_text.split())
        text1, text2 = comment_text.split("[SEP]")

        inputs = self.tokenizer.encode_plus(
            text1,
            text2,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            truncation=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [16]:
# Creating the dataset and dataloader for the neural network

train_x = x
train_y = y

print("FULL Dataset: {}".format(x.shape))
print("TRAIN Dataset: {}".format(train_x.shape))
print("TEST Dataset: {}".format(test_x.shape))
print(f"train_y shape={train_y.shape}")
print(f"test_y shape={test_y.shape}")

training_set = CustomDataset(train_x,train_y, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_x, test_y, tokenizer, MAX_LEN)

FULL Dataset: (83160,)
TRAIN Dataset: (83160,)
TEST Dataset: (9240,)
train_y shape=(83160,)
test_y shape=(9240,)


In [17]:
import pickle
def save_data(obj, fn):
    with open(fn, 'wb') as f:
        pickle.dump(obj, f)
save_data(train_x, "s_train_x.pkl")
save_data(train_y, "s_train_y.pkl")
save_data(test_x, "s_test_x.pkl")
save_data(test_y, "s_test_y.pkl")

In [18]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [19]:
import time


def train(epoch):
    model.train()
    start = time.time()
    for step,data in enumerate(training_loader, 0):
        
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)
        outputs,_ = model(ids, mask, token_type_ids)
        if step == 0:
            for row in ids:
                print(row)
            #print(f"ids={ids}")
            print(f"mask={mask}")
            #print(f"")

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if step%50==0:
            print(f'Epoch: {epoch},step={step}, Loss:  {loss.item()}')
            end = time.time()
            print(f"cost={end-start}")
            start = end
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [20]:
for epoch in range(1):
    train(epoch)

<class 'transformers.modeling_outputs.BaseModelOutputWithPooling'>
BaseModelOutputWithPooling(last_hidden_state=tensor([[[ 1.3934e+00, -4.9442e-02, -7.1780e-01,  ..., -4.0227e-01,
          -1.0844e+00,  6.6136e-02],
         [ 4.4788e-01,  6.8313e-01,  5.5018e-01,  ..., -8.5616e-01,
          -1.1559e-01,  8.1038e-02],
         [ 7.4120e-01,  2.6481e-01, -1.6048e-01,  ..., -3.1493e-01,
           5.6058e-01, -6.8613e-01],
         ...,
         [ 3.6142e-02, -1.4308e-01, -6.0756e-01,  ..., -7.9544e-02,
          -1.6371e-01,  4.9221e-02],
         [ 3.6306e-02, -2.5196e-02, -7.5452e-02,  ...,  2.0647e-01,
           1.4950e-01, -2.4644e-01],
         [-1.2628e-01, -2.0819e-01, -7.8644e-01,  ..., -2.5041e-01,
           2.3834e-01, -3.2076e-01]],

        [[ 5.7467e-01,  1.3777e+00, -5.1000e-01,  ...,  4.2071e-01,
          -2.4156e-01, -7.4867e-02],
         [ 2.2821e-01, -3.0988e-01,  3.3385e-01,  ...,  1.8157e-02,
          -3.9946e-01,  9.6708e-02],
         [ 5.5916e-01, -1.1362e-

ValueError: ignored

In [25]:
from transformers import BertModel, BertConfig

# Initializing a BERT bert-base-uncased style configuration
configuration = BertConfig()
gggg
# Initializing a model from the bert-base-uncased style configuration
model = BertModel(configuration)

# Accessing the model configuration
configuration = model.config

In [27]:
model(ids, mask, token_type_ids)

NameError: ignored